# 2 Layer Neural Network

## Imports

In [1]:
import os
import pandas as pd

## Import data and create .csv

In [2]:
# Directory to MRI Images
base_dir = '/Users/benrandoing/Downloads/archive/AugmentedAlzheimerDataset'
classes = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']

if not os.path.exists('dataset.csv'):
    data = []

    for label, class_name in enumerate(classes):
        class_dir = os.path.join(base_dir, class_name)
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            data.append([image_path, label])

    df = pd.DataFrame(data, columns=['path', 'label'])
    df.to_csv('dataset.csv', index=False)

## Data Loading and Pre-Processing

In [3]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.dataframe = pd.read_csv(csv_file)
        self.dataframe = self.dataframe.sample(frac=1).reset_index(drop=True)  # Shuffle the dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 0]
        image = Image.open(img_name)
        label = int(self.dataframe.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to 224 X 224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    #transforms.Lambda(lambda x: x.view(-1))  # Flatten the image
])

# Load the entire dataset
dataset = CustomDataset(csv_file='dataset.csv', transform=transform)

# Split the dataset into train, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [4]:
print(f'Training Samples: {len(train_dataset)}')
print(f'Validation Samples: {len(val_dataset)}')
print(f'Test Samples: {len(test_dataset)}')

Training Samples: 23788
Validation Samples: 5097
Test Samples: 5099


In [5]:
print(train_dataset)

In [6]:
import matplotlib.pyplot as plt
#plt.plot(train_dataset[0][0])
#plt.show()

## Pytorch Logistic Regression Model

In [7]:
class FiveLayerNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4, output_dim):
        super(FiveLayerNN, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim1)
        self.fc2 = torch.nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = torch.nn.Linear(hidden_dim2, hidden_dim3)
        self.fc4 = torch.nn.Linear(hidden_dim3, hidden_dim4)
        self.fc5 = torch.nn.Linear(hidden_dim4, output_dim)

    def forward(self, x):
        #x = x.view(-1, 224 * 224 * 3)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x

input_dim = 224 * 224 * 3  # Assuming the images are RGB
hidden_dim1 = 512
hidden_dim2 = 256
hidden_dim3 = 128
hidden_dim4 = 64
output_dim = 4  # Four classes

#model = FiveLayerNN(input_dim, hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4, output_dim)
import torchvision
from torchvision.models import resnet18, ResNet18_Weights
model = torchvision.models.resnet18(weights=ResNet18_Weights.DEFAULT)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

model.fc = torch.nn.Linear(model.fc.in_features, output_dim)


## Training Loop

In [8]:
learning_rate = 0.001
num_epochs = 20
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()

import torch.optim
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    i = 0
    for inputs, labels in train_loader:
        i = i + 1
        print("Training continues..", total_loss, "current i = ", i)    
        optimizer.zero_grad()
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)        
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    #lr decay
    #lr_scheduler.step()

    print(f"Iteration {epoch} - Loss = {total_loss / len(train_loader)}")

Training continues.. 0.0 current i =  1
Training continues.. 1.6493802070617676 current i =  2
Training continues.. 3.084267020225525 current i =  3
Training continues.. 4.906838297843933 current i =  4
Training continues.. 6.269284605979919 current i =  5
Training continues.. 7.564918041229248 current i =  6
Training continues.. 9.062071204185486 current i =  7
Training continues.. 10.207144856452942 current i =  8
Training continues.. 11.263141512870789 current i =  9
Training continues.. 12.425619006156921 current i =  10
Training continues.. 13.328285574913025 current i =  11
Training continues.. 14.325457096099854 current i =  12
Training continues.. 15.234297573566437 current i =  13
Training continues.. 16.252736032009125 current i =  14
Training continues.. 17.097197234630585 current i =  15
Training continues.. 17.862726986408234 current i =  16
Training continues.. 18.763486087322235 current i =  17
Training continues.. 19.643886625766754 current i =  18
Training continues.. 

Training continues.. 98.10095390677452 current i =  150
Training continues.. 98.52549636363983 current i =  151
Training continues.. 98.94567632675171 current i =  152
Training continues.. 99.40549185872078 current i =  153
Training continues.. 99.82854878902435 current i =  154
Training continues.. 100.24247962236404 current i =  155
Training continues.. 100.61895605921745 current i =  156
Training continues.. 101.0957045853138 current i =  157
Training continues.. 101.51727795600891 current i =  158
Training continues.. 101.9269460439682 current i =  159
Training continues.. 102.3719770014286 current i =  160
Training continues.. 102.79980990290642 current i =  161
Training continues.. 103.29922443628311 current i =  162
Training continues.. 103.88760095834732 current i =  163
Training continues.. 104.30286258459091 current i =  164
Training continues.. 104.80187922716141 current i =  165
Training continues.. 105.34626644849777 current i =  166
Training continues.. 105.76974779367447

Training continues.. 154.1945916712284 current i =  295
Training continues.. 154.4030936062336 current i =  296
Training continues.. 154.75516375899315 current i =  297
Training continues.. 154.97497045993805 current i =  298
Training continues.. 155.33703717589378 current i =  299
Training continues.. 155.60184398293495 current i =  300
Training continues.. 155.8956361413002 current i =  301
Training continues.. 156.08671697974205 current i =  302
Training continues.. 156.3521222770214 current i =  303
Training continues.. 156.83547028899193 current i =  304
Training continues.. 157.31240290403366 current i =  305
Training continues.. 157.58002203702927 current i =  306
Training continues.. 157.83184170722961 current i =  307
Training continues.. 158.10603988170624 current i =  308
Training continues.. 158.37647932767868 current i =  309
Training continues.. 158.68919748067856 current i =  310
Training continues.. 158.89318783581257 current i =  311
Training continues.. 159.2192965596

Training continues.. 12.57866282388568 current i =  69
Training continues.. 12.63569662719965 current i =  70
Training continues.. 12.832902513444424 current i =  71
Training continues.. 13.073682703077793 current i =  72
Training continues.. 13.215460322797298 current i =  73
Training continues.. 13.530337177217007 current i =  74
Training continues.. 13.686873935163021 current i =  75
Training continues.. 13.956956587731838 current i =  76
Training continues.. 14.14934178441763 current i =  77
Training continues.. 14.320431910455227 current i =  78
Training continues.. 14.478970475494862 current i =  79
Training continues.. 14.66648130863905 current i =  80
Training continues.. 14.765911892056465 current i =  81
Training continues.. 14.958888232707977 current i =  82
Training continues.. 15.255621403455734 current i =  83
Training continues.. 15.512422412633896 current i =  84
Training continues.. 15.898554980754852 current i =  85
Training continues.. 16.128051221370697 current i = 

Training continues.. 38.757688254117966 current i =  215
Training continues.. 38.921478509902954 current i =  216
Training continues.. 39.06730791926384 current i =  217
Training continues.. 39.183981105685234 current i =  218
Training continues.. 39.38151369988918 current i =  219
Training continues.. 39.539204105734825 current i =  220
Training continues.. 39.61725914478302 current i =  221
Training continues.. 39.70147367566824 current i =  222
Training continues.. 39.98460143059492 current i =  223
Training continues.. 40.11978230625391 current i =  224
Training continues.. 40.19418577849865 current i =  225
Training continues.. 40.38784396648407 current i =  226
Training continues.. 40.50512534379959 current i =  227
Training continues.. 40.7152624130249 current i =  228
Training continues.. 40.96019434928894 current i =  229
Training continues.. 41.150940999388695 current i =  230
Training continues.. 41.26973066478968 current i =  231
Training continues.. 41.458214439451694 curr

...

Training continues.. 0.409995343630726 current i =  27
Training continues.. 0.4343946558437892 current i =  28
Training continues.. 0.43459546996018616 current i =  29
Training continues.. 0.4377045199435088 current i =  30
Training continues.. 0.4503096375774476 current i =  31
Training continues.. 0.47433912345877616 current i =  32
Training continues.. 0.521425947190437 current i =  33
Training continues.. 0.6031431477676961 current i =  34
Training continues.. 0.658075075240049 current i =  35
Training continues.. 0.665062508189294 current i =  36
Training continues.. 0.6779287500467035 current i =  37
Training continues.. 0.6948113875623676 current i =  38
Training continues.. 0.7202627366568777 current i =  39
Training continues.. 0.7232239084478351 current i =  40
Training continues.. 0.7247819122785586 current i =  41
Training continues.. 0.7255088772435556 current i =  42
Training continues.. 0.7323332924206625 current i =  43
Training continues.. 0.735521188871644 current i =

Training continues.. 2.000115770577395 current i =  173
Training continues.. 2.0106453389962553 current i =  174
Training continues.. 2.012401669016981 current i =  175
Training continues.. 2.0127543270864408 current i =  176
Training continues.. 2.012816565351386 current i =  177
Training continues.. 2.022922661395569 current i =  178
Training continues.. 2.0237197959868354 current i =  179
Training continues.. 2.0239509236780577 current i =  180
Training continues.. 2.0335316960481578 current i =  181
Training continues.. 2.0340298471637652 current i =  182
Training continues.. 2.0348769629272283 current i =  183
Training continues.. 2.0355010877610766 current i =  184
Training continues.. 2.036754186359758 current i =  185
Training continues.. 2.079297448603029 current i =  186
Training continues.. 2.08047016835917 current i =  187
Training continues.. 2.0809765920057544 current i =  188
Training continues.. 2.0861537522650906 current i =  189
Training continues.. 2.0871647415697225

Training continues.. 4.036919029826095 current i =  318
Training continues.. 4.037639633741492 current i =  319
Training continues.. 4.0521111292655405 current i =  320
Training continues.. 4.08474459032368 current i =  321
Training continues.. 4.0897518088931974 current i =  322
Training continues.. 4.091818212986254 current i =  323
Training continues.. 4.139199923247361 current i =  324
Training continues.. 4.139594292315451 current i =  325
Training continues.. 4.160710349726287 current i =  326
Training continues.. 4.164886213886348 current i =  327
Training continues.. 4.170291621285287 current i =  328
Training continues.. 4.197628201964108 current i =  329
Training continues.. 4.214593187021819 current i =  330
Training continues.. 4.289014473604766 current i =  331
Training continues.. 4.300685470971075 current i =  332
Training continues.. 4.306515448308346 current i =  333
Training continues.. 4.3068858372171235 current i =  334
Training continues.. 4.30910656006381 current 

Training continues.. 1.6269420798053034 current i =  93
Training continues.. 1.6295314885792322 current i =  94
Training continues.. 1.6329512587399222 current i =  95
Training continues.. 1.7018941259593703 current i =  96
Training continues.. 1.7089620303013362 current i =  97
Training continues.. 1.7128955275402404 current i =  98
Training continues.. 1.7324185036704876 current i =  99
Training continues.. 1.7340238215983845 current i =  100
Training continues.. 1.7908208268345334 current i =  101
Training continues.. 1.8001192734227516 current i =  102
Training continues.. 1.8085772116319276 current i =  103
Training continues.. 1.809742977784481 current i =  104
Training continues.. 1.8121909533510916 current i =  105
Training continues.. 1.8126236212847289 current i =  106
Training continues.. 1.81364531032159 current i =  107
Training continues.. 1.830149678135058 current i =  108
Training continues.. 1.8302966372430092 current i =  109
Training continues.. 1.877554470425821 cur

Training continues.. 5.27805226125929 current i =  239
Training continues.. 5.306694220911595 current i =  240
Training continues.. 5.3154060218221275 current i =  241
Training continues.. 5.318508505515638 current i =  242
Training continues.. 5.319084987262613 current i =  243
Training continues.. 5.346006581778056 current i =  244
Training continues.. 5.348381198462448 current i =  245
Training continues.. 5.44508087587019 current i =  246
Training continues.. 5.458503570858738 current i =  247
Training continues.. 5.458799315922079 current i =  248
Training continues.. 5.468237988025066 current i =  249
Training continues.. 5.475334584552911 current i =  250
Training continues.. 5.476336839914438 current i =  251
Training continues.. 5.479882174477098 current i =  252
Training continues.. 5.54844328139734 current i =  253
Training continues.. 5.550375013568555 current i =  254
Training continues.. 5.560074538880144 current i =  255
Training continues.. 5.572626887777005 current i =

## Evaluation

In [9]:
# Validation loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

validation_accuracy = 100 * correct / total
print(f"Validation Accuracy: {validation_accuracy}%")

Validation Accuracy: 90.6219344712576%


In [10]:
# Testing loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy}%")

Test Accuracy: 90.68444793096685%


In [11]:
import torch
torch.save(model.state_dict(), 'resnet18_more_epochs_adam_model_weights.pth')

### Saliency Plots